<a href="https://colab.research.google.com/github/Randyflourish/Intro2AI-Final/blob/main/AI_Final_BASALT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div style="text-align: center">
  <img src="https://github.com/KarolisRam/MineRL2021-Intro-baselines/blob/main/img/colab_banner.png?raw=true">
</div>

# Introduction
This notebook is the installation part for the [MineRL 2022](https://minerl.io/) competition, building on the original introductory notebooks created for the MineRL 2021 competition.

## Note: About this file

This file is updated by NYCU 2024 Spring Intro2AI Team 11: まふまふ.
The original file is come from [here](https://colab.research.google.com/drive/1rJ3lGy-bG7kJRe_wYBWg7fjSaD9oOMDw?usp=sharing)

## There's a video to explain...
Please visit [this intro YouTube video](https://youtu.be/8yIrWcyWGek) to see some background information.  Hopefully, this will lead to a number of additional videos that explore what can be done in this environment...

And if you see me=@mdda online, then please say "Hi!"

## Software 2.0
The approach we are going to use, where we took some human written code and replaced it with an AI component is quite similar to how Tesla approaches self driving cars. See this talk by Andrej Karpathy, Director of AI at Tesla:  
[Building the Software 2.0 Stack](https://databricks.com/session/keynote-from-tesla)


# Setup

In [ ]:
# Env Setup for MineRL
%%capture
!sudo add-apt-repository -y ppa:openjdk-r/ppa
!sudo apt-get purge openjdk-*
!sudo apt-get install openjdk-8-jdk
!sudo apt-get install xvfb xserver-xephyr vnc4server python-opengl ffmpeg
# Takes ~1min to run this
# New Add
!sudo apt-get install -y xvfb  # Install Xvfb

In [ ]:
# Create a local env
import os, sys, time

mine_env = 'mine_env'
mine_env_full = f'/content/{mine_env}'
mine_tar = f'{mine_env}.tar.gz'

if mine_env_full not in sys.path:
  sys.path.insert(0, mine_env_full)
  os.environ['PYTHONPATH'] += f':{mine_env_full}'

mine_env, mine_env_full, mine_tar

In [ ]:
# Connect to Google Drive and build decompress the env dir
# Please press "Yes" to continue the whole project

# Note: Please make sure that the path you put env.tar.gz into your Google Drive is correct, and the pack is latest
# You can check the path in Github readme.md
MINE_ENV_IS_NEW = True

from google.colab import drive  # google.colab contains functions specifically for interacting with Google Colab's environment.
drive.mount('/content/drive')    # mounts your Google Drive as a local file system
if os.path.isfile(f'/content/drive/MyDrive/pythonLib/{mine_tar}'): # check if "mine_env.tar.gz" is in your Google Drive
  ! cp /content/drive/MyDrive/pythonLib/$mine_tar ./$mine_tar  # ! means the command is to be executed in the shell rather than as Python code.
                                              # This command copies the file from your Google Drive to the current working directory of the Colab notebook.

  ! ls -l ./$mine_tar                         # This lists the file details such as permissions, owner, size, and modification date for the copied file in the current directory.
                                              # It helps verify that the file has been copied correctly and shows its properties.
  # e.g.: -rw------- 1 root root 1510118446 Jun 26 08:48 ./mine_env.tar.gz

  # ! tar -tzf ./$mine_tar | grep minerl | head -5    # list some contents of the compressed tar file without extracting it
  ! tar -xzf ./$mine_tar    # This extracts the contents of the tar file into the current directory

  MINE_ENV_IS_NEW = False
  # Takes 1min too (huge saving!)

sys.path.append('/content/drive/MyDrive/pythonLib')
sys.path.append('/content/drive/MyDrive/pythonLib/VPT')

"DONE"

In [ ]:
# Check default packages (execute if needed)
# !pip3 list

In [ ]:
# Should not be executed

# Build the mine_env if necessary
"""
try:
  from pyvirtualdisplay import Display
except :
  !pip3 install --target=$mine_env git+https://github.com/minerllabs/minerl@v1.0.2   # 21 mins
  # https://stackoverflow.com/questions/55833509/attributeerror-type-object-callable-has-no-attribute-abc-registry
  !mv $mine_env/typing.py $mine_env/MEH-typing.py  # Fix for Python3.7 ...

  !pip3 install --target=$mine_env pyvirtualdisplay  # 4 secs  #注 Display creates a virtual framebuffer that graphical applications can use to render output as if they were using a real monitor.
                                                              #注 This allows you to run applications that require a GUI without having an actual GUI environment installed on the system.
  !pip3 install --target=$mine_env --upgrade colabgymrender # 22 secs  #注 colabgymrender provides a workaround by capturing the graphical output of the environment and displaying it within the notebook.

  MINE_ENV_IS_NEW = True
  # NB: some restart notices in the output ... but there's no need to restart!
  #     In any case, please wait for the 'DONE' message to print out
f"DONE, with MINE_ENV_IS_NEW={MINE_ENV_IS_NEW}"
"""

In [ ]:
# Execute this if necessary

# Check content of mine_env
# ! du -b mine_env | tail -5  # mine_env = ~ 2,094,031,775 bytes overall (a little bit less)

In [ ]:
# Execute this if necessary

# Build new env.tar.gz file in google drive
"""
if MINE_ENV_IS_NEW: #  or True
  # ! ls -l /gdrive/MyDrive/mine*
  ! rm -f ./$mine_tar   #注 removes the existing tar.gz archive of the environment, if any, from the current directory.
  ! tar -czf ./$mine_tar $mine_env  #注 This command creates a new compressed (gzipped) tar archive of the directory specified by the $mine_env variable (the environment directory).
  ! ls -l ./$mine_tar
  # Without running the env...
  # -rw-r--r-- 1 root root 1505020174 Jun 26 07:26 ./mine_env.tar.gz
  # Once the minerl env has been reset once (i.e. java has built...)
  # -rw------- 1 root root 1511976116 Jun 26 08:43 ./mine_env.tar.gz
  ! tar -tzf ./$mine_tar | head
  ! cp ./$mine_tar /content/drive/MyDrive/pythonLib/  #注 This copies the newly created archive to a Google Drive directory.
  ! ls -l /content/drive/MyDrive/pythonLib/$mine_tar
"DONE"
"""

# Import Libraries

In [ ]:
import os   # For interacting with the operating system.
import time
import random

import numpy as np  # For numerical operations.

import gym    # To create and manage environments based on the OpenAI Gym toolkit.
import minerl

from tqdm.notebook import tqdm  # For displaying progress bars in Jupyter notebooks.
from colabgymrender.recorder import Recorder # To facilitate rendering of Gym environments in Google Colab.
from pyvirtualdisplay import Display # To create a virtual display to render environments in a headless server or environment like Google Colab.

import logging
logging.disable(logging.ERROR) # reduce clutter, remove if something doesn't work to see the error logs.

np.__version__  # '1.21.6' => that this is reading from our ~/mine_env directory
# Numpy version may be different from the content above
# About warning: since warning is in a local package, so if error occurs, please comment the specific line

import cv2
#from google.colab.patches import cv2_imshow
#from PIL import Image
import matplotlib.pylab as plt

import glob
import json
import torch as th
import torchvision.transforms.functional as TF
from torch import nn
from torch.nn import functional as F
from torch import optim
from run_inverse_dynamics_model import json_action_to_env_action


disp = Display(visible=0, backend="xvfb")
disp.start();

# Download Dataset

In [ ]:
# Download dataset for training
# If you just want to test the agent with exist model and weight, don't execute this

from download_dataset import download_file
download_file(40) # default is 400, about 40 GB, increase the number if you have enough disk

# Construct Inverse Dynamic Model Agent

This is model for predict actions by given a query of frames.

It is not necessary to execute this part if you just want to test the basic model.

In [ ]:
from inverse_dynamics_model import load_IDM_agent
IDMAgent = load_IDM_agent()

In [ ]:
# Test for IDMAgent
from agent import ENV_KWARGS # need to modify
required_resolution = ENV_KWARGS["resolution"]
files = glob.glob("/content/MineRLBasaltFindCave-v0/*.mp4")
video_path = files[0]
json_path = video_path.replace(".mp4", ".jsonl")

cap = cv2.VideoCapture(video_path)
frames = []

json_index = 0
with open(json_path) as json_file:
  json_lines = json_file.readlines()
  json_data = "[" + ",".join(json_lines) + "]"
  json_data = json.loads(json_data)

for _ in range(5000):
  ret, frame = cap.read()
  break
  if not ret:
    break
  assert frame.shape[0] == required_resolution[1] and frame.shape[1] == required_resolution[0], "Video must be of resolution {}".format(required_resolution)
  # BGR -> RGB
  frames.append(frame[..., ::-1])
  break
  if len(frames) == 100 or len(frames) == 50:
    l = len(frames)
    fs = np.stack(frames)
    predicted_actions = IDMAgent.predict_actions(fs)
    for i in range(50):
      env_action, _ = json_action_to_env_action(json_data[json_index])
      json_index += 1
      for y, (action_name, action_array) in enumerate(predicted_actions.items()):
        print(f"{action_name}: {action_array[0, (l - 50 + i)]} ({env_action[action_name]}), ", end = "")
      print("\n")
    frames = frames[50:99]

predicted_actions = IDMAgent.predict_actions(fs)
l = len(frames)
for i in range(50, l):
  env_action, _ = json_action_to_env_action(json_data[json_index])
  json_index += 1
  for y, (action_name, action_array) in enumerate(predicted_actions.items()):
    print(f"{action_name}: {action_array[0, (l - 50 + i)]} ({env_action[action_name]}), ", end = "")
  print("\n")

# Device setup
device = "cuda" if GPU is avalible; otherwise, device = "cpu"

In [ ]:
device = th.device("cuda" if th.cuda.is_available() else "cpu")
print(device)

# Util
including gym env building

In [ ]:
# Build environment, setup some useful variables

# transform of env action and agent action
env = gym.make("MineRLBasaltFindCave-v0")
NOOP = env.action_space.no_op()

# binary encoding of env_action
# forward, back, left, right, sneak, sprint(run), jump, ESC = 2^7, ..., 2^0
ACTION_LIST = ["forward", "back", "left", "right", "sneak", "sprint", "jump", "ESC"]

video_src = glob.glob("/content/MineRLBasaltFindCave-v0/*.mp4")

In [ ]:
# Util declaration

def env_action_to_agent(env_action: dict):
    target_action_C = int(0)
    target_action_R = env_action["camera"]
    for act in ACTION_LIST:
        target_action_C *= 2
        target_action_C += 1 if env_action.get(act) == 1 else 0
    if target_action_C == 0 and np.array_equal(target_action_R, np.zeros(2)):
        isNoop = True
    else:
        isNoop = False
    return [target_action_C, target_action_R, isNoop]

def agent_action_to_env(agent_action_C, agent_action_R):
    target_action = NOOP
    ACTION_LIST_Rev = ACTION_LIST.copy()
    ACTION_LIST_Rev.reverse()
    for act in ACTION_LIST_Rev:
        target_action[act] = 1 if agent_action_C % 2 == 1 else 0
        agent_action_C //= 2  # Use integer division to keep agent_action_C as an integer
    target_action["camera"] = agent_action_R
    return target_action

def img_to_tensor(frames, size):
  target_tensor = th.empty((0, 3, size[1], size[0]), dtype = th.float32)
  for frame in frames:
    frame = cv2.resize(frame, size)
    frame = TF.to_tensor(frame).unsqueeze(0) # Including reshape and normalize
    target_tensor = th.cat((target_tensor, frame), dim = 0)
  return target_tensor

def create_video(frames, path = "/content/drive/MyDrive/AIFinalResult", name = "myVideo"):
  if not os.path.isdir(path):
    os.mkdir(path)
  width = frames[0].shape[1]
  height = frames[0].shape[0]
  fourcc = cv2.VideoWriter_fourcc(*'MP4V')
  out = cv2.VideoWriter(path + "/" + name + ".mp4", fourcc, 20, (width, height))
  for frame in frames:
    out.write(cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))
  out.release()

# Neural Network for Behavior Cloning


In [ ]:
# NN architecture

class SimpleNet(nn.Module):

  def __init__(self, output_size = 256):
    super().__init__()
    self.conv1 = nn.Conv2d(in_channels = 3, out_channels = 64, kernel_size = 10, stride = 5) # 17 * 31
    self.conv2 = nn.Conv2d(in_channels = 64, out_channels = 64, kernel_size = 2, padding = 1) # 17 * 31

    self.fc1 = nn.Linear(in_features = 9 * 16 * 64, out_features = 4096)
    self.fc2 = nn.Linear(in_features = 4096, out_features = output_size)

  def forward(self, input): # 160 * 90 * 3 RGB tensor
    x = F.relu(self.conv1(input))
    x = F.relu(self.conv2(x))
    x = F.max_pool2d(x, kernel_size = 2, stride = 2) # 9 * 16
    x = th.flatten(x, start_dim = 1)
    x = F.relu(self.fc1(x))
    x = self.fc2(x)
    return x # int between [0, 255]

class AlexNet(nn.Module):

  def __init__(self, output_size = 256):
    super().__init__()
    self.conv1 = nn.Conv2d(in_channels = 3, out_channels = 96, kernel_size = 11, stride = 4)
    self.conv2 = nn.Conv2d(in_channels = 96, out_channels = 256, kernel_size = 5, padding = 2)
    self.conv3 = nn.Conv2d(in_channels = 256, out_channels = 384, kernel_size = 3, padding = 1)
    self.conv4 = nn.Conv2d(in_channels = 384, out_channels = 384, kernel_size = 3, padding = 1)
    self.conv5 = nn.Conv2d(in_channels = 384, out_channels = 256, kernel_size = 3, padding = 1)

    self.dropout = nn.Dropout(0.5)

    self.fc1 = nn.Linear(in_features = 256*6*6, out_features = 4096)
    self.fc2 = nn.Linear(in_features = 4096, out_features = 4096)
    self.fc3 = nn.Linear(in_features = 4096, out_features = output_size)

  def forward(self, input): # 227 * 227 * 3 RGB tensor
    x = F.relu(self.conv1(input))
    x = F.max_pool2d(x, kernel_size = 3, stride = 2)
    x = F.relu(self.conv2(x))
    x = F.max_pool2d(x, kernel_size = 3, stride = 2)
    x = F.relu(self.conv3(x))
    x = F.relu(self.conv4(x))
    x = F.relu(self.conv5(x))
    x = F.max_pool2d(x, kernel_size = 3, stride = 2)
    x = th.flatten(x, start_dim = 1)
    x = F.relu(self.fc1(x))
    x = self.dropout(x)
    x = F.relu(self.fc2(x))
    x = self.dropout(x)
    x = self.fc3(x)
    return x # int between [0, 255]


# FindCave Agent

In [ ]:
# Agent architecture

class FindCaveAgent():

  def __init__(self, learning_rate = 0.001, NetC = "", NetR = ""):

    # For Classification (8 button)
    if NetC == "AlexNet":
      self.policyC = AlexNet(output_size = 256).to(device)
      self.sizeC = (227, 227)
    else:
      self.policyC = SimpleNet(output_size = 256).to(device)
      self.sizeC = (160, 90)
    self.optimizerC = optim.SGD(self.policyC.parameters(), lr = learning_rate)
    self.lossCFunc = nn.CrossEntropyLoss()

    # For regression (camera)
    if NetC == "AlexNet":
      self.policyR = AlexNet(output_size = 2).to(device)
      self.sizeR = (227, 227)
    else:
      self.policyR = SimpleNet(output_size = 2).to(device)
      self.sizeR = (160, 90)
    self.optimizerR = optim.SGD(self.policyR.parameters(), lr = learning_rate)
    self.lossRFunc = nn.MSELoss()

  def train(self, batch_size = 32, epoch = 10):
    # Training function

    for _ in range(epoch):

      vcaps = []
      vlength = []
      action_data = []
      action_index = 0
      counter = 0
      random_video = random.sample(video_src, batch_size)

      for x in random_video:
        vcaps.append(cv2.VideoCapture(x))
        vlength.append(int(vcaps[-1].get(7)) - 1)
        with open(x.replace(".mp4", ".jsonl")) as json_file:
          json_lines = json_file.readlines()
          json_data = "[" + ",".join(json_lines) + "]"
          json_data = json.loads(json_data)
          action_data.append(json_data)

      while len(vcaps) != 0:
        counter += 1
        if counter % 20 == 0:
          print(counter)
        if action_index == 3600:
          break
        frames = []
        actionsC = []
        actionsR = []
        pop_cap = []

        for i in range(batch_size):
          if action_index < vlength[i]:
            actions = env_action_to_agent(json_action_to_env_action(action_data[i][action_index])[0])
            if actions[2] == True:
              vcaps[i].read()
              continue
            frames.append(vcaps[i].read()[1])
            actionsC.append(actions[0])
            actionsR.append(actions[1])
          else:
            frames.append(vcaps[i].read()[1])
            actionsC.append(1)
            actionsR.append([0, 0])
            pop_cap.append(i)

        if len(pop_cap) > 0:
          pop_cap.reverse()
          for i in pop_cap:
            vcaps[i].release()
            vcaps.pop(i)
            vlength.pop(i)
            action_data.pop(i)
            batch_size -= 1

        action_index += 1

        if len(actionsC) == 0:
          continue

        frames_tensor = img_to_tensor(np.array(frames), self.sizeC).to(device)
        actionC_tensor = th.LongTensor(np.array(actionsC)).to(device)
        resultC = self.policyC(frames_tensor)
        frames_tensor = img_to_tensor(np.array(frames), self.sizeR).to(device)
        actionR_tensor = th.FloatTensor(np.array(actionsR)).to(device)
        resultR = self.policyR(frames_tensor)

        self.optimizerC.zero_grad()
        lossC = self.lossCFunc(resultC, actionC_tensor)
        lossC.backward()
        if counter % 20 == 0: print(lossC)
        self.optimizerC.step()

        self.optimizerR.zero_grad()
        lossR = self.lossRFunc(resultR, actionR_tensor)
        lossR.backward()
        if counter % 20 == 0: print(lossR)
        self.optimizerR.step()

    self.save_model_weights()

  def test(self, test_video = 5):

    random.seed(1919810)
    random_video = random.sample(video_src, test_video)
    counter = 1

    # TODO
    for video in random_video:

      counter += 1
      vcap = cv2.VideoCapture(video)
      vlength = vcap.get(7) - 1
      action_data = []
      with open(video.replace(".mp4", ".jsonl")) as json_file:
        json_lines = json_file.readlines()
        json_data = "[" + ",".join(json_lines) + "]"
        json_data = json.loads(json_data)
        action_data.append(json_data)
      action_index = 0

      accum_lossC = 0
      accum_lossR = 0

      for i in tqdm(range(vlength), desc = "Processing " + str(counter)):

        frame = []
        actionC = []
        actionR = []

        frame.append(vcap[i].read())
        actions = env_action_to_agent(json_action_to_env_action(action_data[action_index])[0])
        action_index += 1
        if actions[2] == True:
          continue
        actionC.append(actions[0])
        actionR.append(actions[1])

        frame_tensor = img_to_tensor(np.array(frame), self.sizeC).to(device)
        actionC_tensor = th.LongTensor(np.array(actionC)).to(device)
        resultC = self.policyC(frame_tensor)
        frame_tensor = img_to_tensor(np.array(frame), self.sizeR).to(device)
        actionR_tensor = th.FloatTensor(np.array(actionR)).to(device)
        resultR = self.policyR(frame_tensor)

        lossC = self.lossCFunc(resultC, actionC_tensor)
        lossR = self.lossRFunc(resultR, actionR_tensor)
        accum_lossC += lossC
        accum_lossR += lossR

      print(f"Video {counter} lossC: {accum_lossC}, lossR: {accum_lossR}")

  def predict(self, observe):
    # Predict the next action by observation (image)

    with th.no_grad():

      obs_tensor = img_to_tensor([observe, ], self.sizeC).to(device)
      resultC = self.policyC(obs_tensor).squeeze().argmax().cpu().numpy()
      obs_tensor = img_to_tensor([observe, ], self.sizeR).to(device)
      resultR = self.policyR(obs_tensor).squeeze().cpu().numpy()
      env_action = agent_action_to_env(resultC, resultR)
      del obs_tensor, resultC, resultR

    return env_action

  def save_model_weights(self, path="minerl_weights.pth"):
    # Save the state dictionaries of models and optimizers
    th.save({
        'policyC_state_dict': self.policyC.state_dict(),
        'optimizerC_state_dict': self.optimizerC.state_dict(),
        'policyR_state_dict': self.policyR.state_dict(),
        'optimizerR_state_dict': self.optimizerR.state_dict(),
    }, path)

  def load_model_weights(self, path="minerl_weights.pth"):
    # Load the state dictionaries of models and optimizers
    checkpoint = th.load(path)
    self.policyC.load_state_dict(checkpoint['policyC_state_dict'])
    self.optimizerC.load_state_dict(checkpoint['optimizerC_state_dict'])
    self.policyR.load_state_dict(checkpoint['policyR_state_dict'])
    self.optimizerR.load_state_dict(checkpoint['optimizerR_state_dict'])

In [ ]:
# Create and Train Agent
TA = FindCaveAgent(NetC = "SimpleNnet", NetR = "SimpleNet")
TA.train()

# TO Train
"""
1. lr = 0.001, NetC = NetR = "SimpleNet"
2. lr = 0.0001, NetC = NetR = "SimpleNet"
3. lr = 0.00055, NetC = NetR = "SimpleNet"
4. lr = 0.001, NetC = NetR = "AlexNet"
5. lr = 0.0001, NetC = NetR = "AlexNet"
6. lr = 0.00055, NetC = NetR = "AlexNet"
"""

# Load Model

In [ ]:
path = "/content/drive/MyDrive/minerl_weights.pth"
TA.load_model_weights(path)

# Testing

In [ ]:
def testing(agent, env, render = False, video = False, seed = 114514):

    env.seed(seed)
    obs = env.reset()
    pov = obs["pov"]

    img_buffer = []

    done = False
    cumulative_reward = 0

    while not done:
      ac = agent.predict(pov)
      obs, reward, done, info = env.step(ac)
      pov = obs["pov"]

      cumulative_reward += reward

      if render:
        plt.imshow(pov)
        plt.show()
        plt.close()  # Important to reduce the usage of RAM

      if video:
        img_buffer.append(cv2.resize(pov, (480, 270))) # Can adjust the size of frame to reduce RAM, original size is (640, 360)

    if video:
      create_video(img_buffer)
    del img_buffer

    print(f"Total Cumulative Reward: {cumulative_reward}")
    return cumulative_reward


render = False
video = False
num_runs = 10
cumulative_rewards = []

for _ in range(num_runs):
    cumulative_reward = testing(TA, env, render = render, video = video)
    cumulative_rewards.append(cumulative_reward)

average_cumulative_reward = sum(cumulative_rewards) / num_runs
print(f"Average Cumulative Reward over {num_runs} runs: {average_cumulative_reward}")


# Other Function

In [ ]:
# Show the format of action
# env.action_space.sample().keys()

In [ ]:
# Have a look at a few actions we might do:
# for _ in range(10):
#   print( env.action_space.sample() )

In [ ]:
# Now that Steve has been spawned, do some actions...
t0 = time.time()
# Env Seed Setting
env.seed(1919810)
obs = env.reset()
pov = obs["pov"]
print(f"{(time.time()-t0):.2f}sec for env.reset")

done, iter = False, 0
actionClist = [128, 128, 128, 130, 130, 130, 130, 130, 130, 128, 128, 0, 0, 0, 1]

img_buffer = []
while not done:
    ac = agent_action_to_env(128, [0, +10])
    if iter > 300:
      ac = agent_action_to_env(1, [0, 0])
    # ac = TA.predict(pov)  # Use this to test the performance of NN
    # Spin around to see what is around us
    # ac["camera"] = [0, +30]  # (pitch, yaw) deltas in degrees : +30 => turn to right

    t1=time.time()
    obs, reward, done, info = env.step(ac)
    # print(obs, reward, info)  # NB: Yikes : obs is only the image!
    # obs = Dict(pov:Box(low=0, high=255, shape=(360, 640, 3)))
    # print(pov.shape) # (360, 640, 3)  Image spec agrees with docs!
    # print(f"{(time.time()-t1):.2f}sec for env.step")  # Approx 0.25sec per step

    pov = obs["pov"]
    img_buffer.append(cv2.resize(pov, (320, 180))) # Can choose not to resize (About 2GB - 4GB RAM consume)

    # env.render()  # This does an internal cv2.imshow that colab rejects
    # cv2_imshow(pov[:, :, ::-1])
    # cv2.waitKey(1)

    # Show image in terminal
    # plt.imshow(pov)
    # plt.show()
    # plt.clf()  # important to reduce the usage of RAM
    iter +=1

create_video(img_buffer, name = "testVideo3")
# path: target dir (default is AIFinalResult in drive, if no such dir, this func will make a new one), name: video name
del img_buffer

# plt.close()

f"{(time.time()-t0):.2f}sec for whole spin"

# Endding

In [ ]:
env.close()
disp.stop()